# Vitessce Widget Tutorial

# Visualization of SpatialData Object

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from urllib.request import urlretrieve
import dask

dask.config.set({'dataframe.query-planning-warning': False})

from spatialdata import read_zarr
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    CoordinationLevel as CL,
    AbstractWrapper,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)
import zipfile


In [ ]:
zip_filepath = Path("data/visium.spatialdata.zarr.zip")
spatialdata_filepath = zip_filepath.with_suffix('')
if not zip_filepath.exists():
    spatialdata_filepath.parent.mkdir(exist_ok=True)
    urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_associated_xenium_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(spatialdata_filepath.parent)
        (spatialdata_filepath.parent / "data.zarr").rename(spatialdata_filepath)
        (spatialdata_filepath / "tables").rename(spatialdata_filepath / "table") # TODO: remove once fixed in Vitessce

## 3. Load the data

Note: this function may print a `FutureWarning`

In [ ]:
spatialdata = read_zarr(spatialdata_filepath)

In [ ]:
spatialdata

In [ ]:
spatialdata.shapes

## 4. Create the Vitessce widget configuration

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

### 4.1. Instantiate a `VitessceConfig` object

Use the `VitessceConfig` constructor to create an instance.

In [ ]:
vc = VitessceConfig(schema_version="1.0.16", name='Visium SpatialData Demo (visium_associated_xenium_io)', description='From https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html')

### 4.2. Add a dataset to the `VitessceConfig` instance

In Vitessce, a dataset is a container for one file per data type. The `.add_dataset(name)` method on the `vc` instance sets up and returns a new dataset instance.

Then, we can call the dataset's `.add_object(wrapper_object)` method to attach a "data wrapper" instance to our new dataset. For example, the `AnnDataWrapper` helps to configure AnnData Zarr stores for use in the Vitessce configuration.

Dataset wrapper classes may require additional parameters to resolve ambiguities. For instance, `AnnData` objects may store multiple clusterings or cell type annotation columns in the `adata.obs` dataframe. We can use the parameter `obs_set_paths` to tell Vitessce that certain columns of the `obs` dataframe correspond to cell type annotations or cell clusterings.

In [ ]:
wrapper = SpatialDataWrapper(
    spatialdata_path=spatialdata_filepath.absolute(),
    image_path="images/CytAssist_FFPE_Human_Breast_Cancer_full_image",
    obs_feature_matrix_path = "table/table/X",
    feature_labels_path = "table/table/var/gene_ids",
    shapes_path = "shapes/CytAssist_FFPE_Human_Breast_Cancer",
    # obs_set_paths = ["table/table/obs/region"],
    coordination_values={"obsType":"spot"}
)

In [ ]:
dataset = vc.add_dataset(name='Breast Cancer Visium').add_object(wrapper)

In [ ]:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
# The below is trying to get the spots to appear.....delete maybe?
# [feature_selection, obs_color_encoding] = vc.add_coordination(ct.FEATURE_SELECTION, ct.OBS_COLOR_ENCODING);
# obs_color_encoding.set_value('geneSelection')
# vc.link_views_by_dict(
#     [spatial, layer_controller],
#     {
#         ct.FEATURE_SELECTION: feature_selection, ct.OBS_COLOR_ENCODING: obs_color_encoding
#     },
#     scope_prefix="init_A_obsSpots"
# )
vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        'photometricInterpretation': 'RGB',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))
vc.link_views([spatial, layer_controller, feature_list], ['obsType'], ['spot'])
# feature_list.use_coordination(feature_selection)
# feature_list.use_coordination(obs_color_encoding)

### 4.4. Define the visualization layout

The `vc.layout(view_concat)` method allows us to specify how our views will be arranged in the layout grid in the widget. The `|` and `/` characters are magic syntax for `hconcat(v1, v2)` and `vconcat(v1, v2)`, respectively.

In [ ]:
vc.layout(spatial | (feature_list / layer_controller));

In [ ]:
vc.to_dict('localhost:8080')

In [ ]:
vw = vc.widget()
vw

In [ ]:
vw.close()


In [ ]:
vw.config